In [ ]:
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as tfl


In [ ]:
def one_hot_encoding(C,Y):
    """
    Input:
    C: number of classes
    Y: 2D label vector specifying the class of respective training example

    Output:
    2D one hot encoding 
    """
    return np.eye(C)[Y.reshape(-1)]

In [ ]:
dataset_train = h5py.File("./content/drive/MyDrive/signDataset/train_signs.h5")
dataset_test = h5py.File("./content/drive/MyDrive/signDataset/test_signs.h5")

#exploring the dataset
print("keys: ", dataset_train.keys())
print(dataset_train['list_classes'][:])

In [ ]:
X_train_orig = np.array(dataset_train["train_set_x"])
y_train_orig = np.array(dataset_train["train_set_y"])
X_test_orig = np.array(dataset_test["test_set_x"])
y_test_orig = np.array(dataset_test["test_set_y"])

print("shapes: ", X_train_orig.shape, y_train_orig.shape, X_test_orig.shape, y_test_orig.shape)

In [ ]:
#preprocessing inputs

#briging y to the desired output shape, a column vector
Y_train = y_train_orig.reshape((y_train_orig.shape[0],1))
Y_test = y_test_orig.reshape((y_test_orig.shape[0],1))
print("new shape: ", Y_train.shape, Y_test.shape)


#normalising the rgb images
X_train = X_train_orig/255
X_test = X_test_orig/255

#since y has multiple labels, we shall use one hot encoding
Y_train = one_hot_encoding(6,Y_train)
Y_test = one_hot_encoding(6,Y_test)
print("new shape: ", Y_train.shape, Y_test.shape)
#previously Y was stored as 0,1,2...5, now it is stored as a vector 


In [ ]:
#visualising the input
index = 108
print("y = ", Y_train[index])
plt.imshow(X_train[index])



In [ ]:
def signModel(input_shape):
    """original"""
    input_img = tf.keras.Input(input_shape)
    Z1 = tfl.Conv2D(filters=8, kernel_size=(4,4), strides=(1,1), padding="same")(input_img)
    A1 = tfl.ReLU()(Z1)
    P1 = tfl.MaxPooling2D(pool_size=(8,8), strides=(8,8), padding="same")(A1)
    Z2 = tfl.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1), padding="same")(P1)
    A2 = tfl.ReLU()(Z2)
    P2 = tfl.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding="same")(A2)
    F = tfl.Flatten()(P2)
    outputs = tfl.Dense(units=6, activation="softmax")(F)
    model = tf.keras.Model(inputs=input_img, outputs = outputs)
    return model

In [ ]:
sign_model = signModel((64,64,3))
sign_model.compile(optimizer='adam',  loss='categorical_crossentropy',
                  metrics=['accuracy'])
sign_model.summary()


In [ ]:
history = sign_model.fit(X_train, Y_train, epochs=100, batch_size=64)

In [ ]:
sign_model.evaluate(X_test, Y_test)

The model has high bias and variance, let us introduce batch normalisation and see what happens, it should facilitate training and decrease bias

In [ ]:
def signModel2(input_shape):
    """original + BN"""
    input_img = tf.keras.Input(input_shape)
    Z1 = tfl.Conv2D(filters=8, kernel_size=(4,4), strides=(1,1), padding="same")(input_img)
    B1 = tfl.BatchNormalization(axis=3)(Z1)
    A1 = tfl.ReLU()(B1)
    P1 = tfl.MaxPooling2D(pool_size=(8,8), strides=(8,8), padding="same")(A1)
    Z2 = tfl.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1), padding="same")(P1)
    B2 = tfl.BatchNormalization(axis=3)(Z2)
    A2 = tfl.ReLU()(B2)
    P2 = tfl.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding="same")(A2)
    F = tfl.Flatten()(P2)
    outputs = tfl.Dense(units=6, activation="softmax")(F)
    model = tf.keras.Model(inputs=input_img, outputs = outputs)
    return model

In [ ]:
sign_model2 = signModel2((64,64,3))
sign_model2.compile(optimizer='adam',  loss='categorical_crossentropy',
                  metrics=['accuracy'])
sign_model2.fit(X_train, Y_train, epochs=100, batch_size=64)
sign_model2.evaluate(X_test, Y_test)

Now the model has high variance, let us introduce regularization

In [ ]:
def signModel3(input_shape):
    """original + BN"""
    input_img = tf.keras.Input(input_shape)
    drate = 0.3
    Z1 = tfl.Conv2D(filters=8, kernel_size=(4,4), strides=(1,1), padding="same")(input_img)
    B1 = tfl.BatchNormalization(axis=3)(Z1)
    A1 = tfl.ReLU()(B1)
    P1 = tfl.MaxPooling2D(pool_size=(8,8), strides=(8,8), padding="same")(A1)
    Z2 = tfl.Conv2D(filters=16, kernel_size=(2, 2), strides=(1, 1), padding="same")(P1)
    B2 = tfl.BatchNormalization(axis=3)(Z2)
    A2 = tfl.ReLU()(B2)
    F = tfl.Flatten()(A2)
    outputs = tfl.Dense(units=6, activation="softmax")(F)
    model = tf.keras.Model(inputs=input_img, outputs = outputs)
    return model

In [ ]:
sign_model3 = signModel3((64,64,3))
sign_model3.compile(optimizer='adam',  loss='categorical_crossentropy',
                  metrics=['accuracy'])

sign_model3.fit(X_train, Y_train, batch_size=32,epochs=25)
sign_model3.evaluate(X_test, Y_test)

In [ ]:
# Assuming 'model' is your trained model, 'X_test' is your test data, and 'Y_test' are the true labels
# Assuming 'predictions' is your array of predicted probabilities
predictions = sign_model3.predict(X_test)

# Convert probabilities to one-hot encoded labels
one_hot_predictions = np.eye(predictions.shape[1])[np.argmax(predictions, axis=1)]

# Find the indices of the incorrect predictions
incorrects = np.where(~np.all(one_hot_predictions == Y_test, axis=1))[0]

# Print the indices of the incorrect predictions
print("Indices of incorrect predictions: ", incorrects)

In [ ]:
# Get the first 20 incorrect indices
first_20_incorrects = incorrects[:20]

fig, axes = plt.subplots(2, 2, figsize=(5, 5))

for i, incorrect in enumerate(first_20_incorrects):
    ax = axes[i//5, i%5]
    ax.imshow(X_test[incorrect], cmap='gray')
    ax.set_title(f"Predicted: {np.argmax(one_hot_predictions[incorrect])}, Actual: {np.argmax(Y_test[incorrect])}")
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
sign_model3.save('model98.keras', overwrite=False)

In [ ]:
sign_model4 = tf.keras.models.load_model('model98.keras')
sign_model4.evaluate(X_test, Y_test)